런타임 > 런타임 유형 변경 > TPU v2-8으로 변경 후 코드 실행

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Clone our code from GitHub if it does not exist.

In [4]:
import os
dataset_name = 'test'
if not os.path.exists('/content/drive/MyDrive/DS535'):
  os.mkdir('/content/drive/MyDrive/DS535')
os.chdir('/content/drive/MyDrive/DS535')
if not os.path.exists(dataset_name):
  os.mkdir(dataset_name)
os.chdir(dataset_name)

In [5]:
if not os.path.exists('DS535_Team3'):
  !git clone https://github.com/anminhyeong/DS535_Team3.git

Cloning into 'DS535_Team3'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 117 (delta 65), reused 58 (delta 25), pack-reused 0 (from 0)
Receiving objects: 100% (117/117), 35.20 KiB | 924.00 KiB/s, done.
Resolving deltas: 100% (65/65), done.


In [6]:
os.chdir('DS535_Team3')

Install packages.

In [9]:
!pip install -U "jax[tpu]==0.4.23" -f https://storage.googleapis.com/jax-releases/libtpu_releases.html
!pip install scipy==1.11.2
!pip install torch_geometric==2.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 38.0 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 10.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910447 sha256=3f2da7172796b51780b58b94073291b8e4cd2b0209a7ce37718bd875fab88bcb
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


We conducted experiments differentiating the following settings:
1. Random walk strategies (random walk, DFS-like walk, highly DFS-like walk)
  - Configured as argument (p_value and q_value) in the preprocess.py command.
  - Random walk: p_value = 1.0, q_value = 1.0
  - DFS-like walk: p_value = 1e10, q_value = 0.5
  - Highly DFS-like walk: p_value = 1e10, q_value = 0.1
2. Mask data type (int, bool)
  - Configured as argument (bool_mask) in the preprocess.py command.
3. Number of hops (num_hops = 3, 4, 5)
  - Configured as argument (num_hops) in the train_{pixel, sbm, zinc}.py command.

Instead of executing preprocess.py in Colab, we executed it in local environment and uploaded the dataset files (npy and npz) to the corresponding folder given below.
- MNIST: ./data/MNIST
- CIFAR10: ./data/CIFAR10
- CLUSTER: ./data/CLUSTER
- PATTERN: ./data/PATTERN
- ZINC: ./data/ZINC/subset

Or you can download the dataset from the following link.

https://drive.google.com/drive/folders/12kKq-WAer7TYh3R5H3wyxzZaWTqzyOyD?usp=drive_link

In [ ]:
# !python preprocess.py --length 5 --num 5 --p_value 1.0 --q_value 1.0 --name MNIST

To apply a boolean mask when running the train_{pixel, sbm, zinc}.py command, you need to include the --bool_mask argument in your command.

In [ ]:
# !python train_pixel.py --name MNIST --num_layers 4 --num_hops 3 --dim_h 128 --dim_v 96
# !python train_pixel.py --name CIFAR10 --num_layers 8 --num_hops 5 --dim_h 96 --dim_v 64
# !python train_sbm.py --name CLUSTER --num_layers 16 --dim_h 64 --dim_v 64 --weight_decay 0.2 --r_min 0.9
# !python train_sbm.py --name PATTERN --num_layers 10 --dim_h 72 --dim_v 64 --weight_decay 0.1 --r_min 0.5
!python train_zinc.py --bool_mask

Namespace(num_layers=11, num_hops=5, dim_h=72, dim_v=72, r_min=0.9, r_max=1.0, max_phase=6.28, drop_rate=0.2, expand=1, act='full-glu', bool_mask=True, lr_min=1e-07, lr_max=0.001, weight_decay=0.1, lr_factor=1.0, epochs=2000, batch_size=32, warmup=0.05, seed=0, gpu='0', length_walk=5, num_walk=5, p_value=1.0, q_value=1.0)
/content/drive/MyDrive/DS535/test/DS535_Team3/train_zinc.py:60: DeprecationWarning: jax.random.KeyArray is deprecated. Use jax.Array for annotations, and jax.dtypes.issubdtype(arr.dtype, jax.dtypes.prng_key) for runtime detection of typed prng keys (i.e. keys created with jax.random.key).
For more information, see https://jax.readthedocs.io/en/latest/jep/9263-typed-keys.html
  key: random.KeyArray
E1220 07:11:28.438127977    8215 oauth2_credentials.cc:176]            Call to http server ended with error 404 [].
# parameters: 483049
train size: 10000; # train steps per epoch: 312; # train steps total: 624000
val size: 1000; # val steps: 32
test size: 1000; # test steps